# Desktop Search Exploratory Analysis - August 2020
[Task](https://phabricator.wikimedia.org/T259766)

The Web team is making serveral changes to search as part of the Desktop Improvements Project. These changes include moving the location of the search in the header and changes the search widget using vue.js. Prior to making those changes, the team is interested in how search is currently used on desktop to inform the design of these new seach changes.

Data comes from the [SearchSatisfaction](https://meta.wikimedia.org/wiki/Schema:SearchSatisfaction) schema, which includes all search events on desktop. Reviewed the time period from 01 January 2020 to 13 August 2020. Any bots or users currently in a subtest were removed from the reviewed data.

# Metrics

- what percentage of submitted searches are submitted via the button (search icon)?
- how often is the button clicked with the search box empty?
- How often do people enter text in the search box versus the advanced page search main input?

# What percentage of searches are submitted via the button?

## Approach

Searches submitted using the search button (not clicking an autcomplete result in the dropdown menu) are logged as action: "click" and position: -1. Note: This also includes users who type and press enter since current instrumentation does not differentiate between these two actions.

Searches submitted by clicking an autcomplete result in the dropdown menu are logged as action: click: and position >=0.


In [153]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); library(wmfdata)
})

## What percent of all search clicks are submitted via the button?

In [110]:

query <- "
SELECT
    SUM(CAST(event.position = -1 AS INT)) AS search_button_clicks,
    SUM(CAST(event.position >= 0 AS INT)) AS autocomplete_clicks,
    COUNT(*) AS all_search_clicks
FROM event.searchSatisfaction
WHERE 
    year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13))
--isolate to search click events 
    AND event.action = 'click' 
-- remove bots
    AND useragent.is_bot = false 
-- remove users in any sub test
    AND event.subTest IS NULL
"

In [111]:
search_events_bysubmissiontype <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [113]:
search_events_prop_bysubmissiontype <- search_events_bysubmissiontype %>%
    gather(submission_type, num_events, 1:3) %>%
    mutate(prop_clicks = num_events/search_events_bysubmissiontype$all_search_clicks * 100) 

search_events_prop_bysubmissiontype

submission_type,num_events,prop_clicks
<chr>,<int>,<dbl>
search_button_clicks,86241201,48.38048
autocomplete_clicks,92014979,51.61952
all_search_clicks,178256180,100.00000


From January through August 13, 2020, a little under half (**48.38%**) of all search submissions using the search widget are completed by clicking on the search icon (or pressing enter). The remainder of all search submissions are completed by selecting one of the results in the drop down menu. 


## What percent of search submission sessions include a search button click?

In [114]:

query <-
"
SELECT
    SUM(CAST(search_click = true AS INT)) AS search_button_click_sessions,
    SUM(CAST(autocomplete_click = true AS INT)) AS autcomplete_click_sessions,
    COUNT(search_session) AS all_click_sessions
FROM
(
SELECT
    SUM(CAST(event.position = -1 AS INT)) >= 1 AS search_click,
    SUM(CAST(event.position >= 0 AS INT)) >= 1 AS autocomplete_click,
    event.searchsessionID AS search_session
FROM event.searchSatisfaction
WHERE 
     year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13))
-- review only sessions where a search was submitted
    AND event.action = 'click' 
-- Remove bots
    AND useragent.is_bot = false 
    AND event.subTest IS NULL 
GROUP BY event.searchsessionID
) AS search
"

In [115]:
search_sessions_bysubmissiontype <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [116]:
search_sessions_bysubmissiontype

search_button_click_sessions,autcomplete_click_sessions,all_click_sessions
<int>,<int>,<int>
49306614,54168267,85899868


In [118]:
search_sessions_prop_bysubmissiontype <- search_sessions_bysubmissiontype %>%
    gather(submission_type, num_sessions, 1:3) %>%
    mutate(prop_click_sessions = num_sessions/search_sessions_bysubmissiontype$all_click_sessions * 100) 

search_sessions_prop_bysubmissiontype

submission_type,num_sessions,prop_click_sessions
<chr>,<int>,<dbl>
search_button_click_sessions,49306614,57.40010
autcomplete_click_sessions,54168267,63.05978
all_click_sessions,85899868,100.00000


57.4% of all search click sessions included a click on the search button.  

Note: Some search sessions can include both a search conducted by submitting the form and by selecting an autcomplete result, which is why the number of sessions for each adds up to more than the total.

# How often is the button clicked with the search box empty?

## Approach

A search session does not start (and data not recorded) until a user starts to enter text into the search widget. We do not record users who click the search button with a blank text box if they have not already started a search session. After starting a session begins, if a user presses the search button again (or presses enter) with the search box empty, they are taken to the Advanced Search Page with no results shown. This is recorded in teh data sets as action: "visitPage", position: -1, and query is NULL. 


## What percent of events are clicked with the search box empty

## What percent of all search events are blank search page events?

In [ ]:


query <-
"
-- find sessions that clicked the search button without text
WITH blank_input_sessions AS (
SELECT 
    DISTINCT event.searchsessionID as session_id_noinput
FROM event.searchSatisfaction
WHERE
    event.position = -1
    AND event.action = 'visitPage'
    AND event.query IS NULL
    AND event.source = 'autocomplete'
    AND year = 2020 AND month = 08 AND day = 03
    AND useragent.is_bot = false 
    AND event.subTest IS NULL 
)
-- find all submitted searches - event.action = 'click'
SELECT
    SUM(CAST(session_id_noinput is not NULL AS int)) AS blank_input_clicks,
    COUNT(*) AS all_search_button_clicks
    FROM
    event.searchSatisfaction 
    LEFT JOIN blank_input_sessions ON event.searchsessionID = blank_input_sessions.session_id_noinput
    WHERE
    year = 2020 AND month = 08 AND day = 03
    AND useragent.is_bot = false 
    AND event.subTest IS NULL 
-- isolate to only searches submitted via the button or pressing enter
    AND event.position = -1
    AND event.action = 'click'
"



In [54]:
noinput_search_clicks <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [121]:
noinput_search_clicks_prop <- noinput_search_clicks  %>%
    mutate(prop_search_events = blank_search_event/all_search_events * 100)

noinput_search_clicks_prop

blank_search_event,all_search_events,prop_search_events
<int>,<int>,<dbl>
10559690,1114299211,0.947653


Searches made with the search box empty account for only 0.94% of all search events. 

## What percent of all search sessions included a no text input search?

In [137]:
query <-
"
WITH blank_input_sessions AS (
SELECT 
    DISTINCT event.searchsessionID as session_id_noinput
FROM event.searchSatisfaction
WHERE
    event.position = -1
    AND event.action = 'visitPage'
    AND event.query IS NULL
    AND event.source = 'autocomplete'
    AND year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13))
    AND useragent.is_bot = false 
    AND event.subTest IS NULL 
)
-- Main Query --
SELECT
    SUM(CAST(session_id_noinput is not NULL AS int)) AS blank_search_sessions,
    COUNT(all_session_id) AS all_search_sessions
FROM (
    SELECT
        event.searchsessionID as all_session_id,
        session_id_noinput
    FROM
    event.searchSatisfaction AS ss
    LEFT JOIN blank_input_sessions ON event.searchsessionID = blank_input_sessions.session_id_noinput
    WHERE
    year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13))
    AND useragent.is_bot = false 
    AND event.subTest IS NULL 
    GROUP BY event.searchsessionID,
    session_id_noinput
) all_sessions
"

In [138]:
noinput_search_sessions <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [139]:
noinput_search_sessions_prop <- noinput_search_sessions   %>%
    mutate(prop_search_sessions = blank_search_sessions/all_search_sessions * 100)

noinput_search_sessions_prop

blank_search_sessions,all_search_sessions,prop_search_sessions
<int>,<int>,<dbl>
5094097,92000595,5.537026


## What percent of click button search sessions included a no text input search ?

In [141]:

query <-
"
WITH blank_input_sessions AS (
SELECT 
    DISTINCT event.searchsessionID as session_id_noinput
FROM event.searchSatisfaction
WHERE
    event.position = -1
    AND event.action = 'visitPage'
    AND event.query IS NULL
    AND event.source = 'autocomplete'
    AND year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13))
    AND useragent.is_bot = false 
    AND event.subTest IS NULL 
)
-- Main Query --
SELECT
    SUM(CAST(session_id_noinput is not NULL AS int)) AS blank_search_sessions,
    COUNT(all_session_id) AS all_search_click_sessions
FROM (
    SELECT
        event.searchsessionID as all_session_id,
        session_id_noinput
    FROM
    event.searchSatisfaction AS ss
    LEFT JOIN blank_input_sessions ON event.searchsessionID = blank_input_sessions.session_id_noinput
    WHERE
    year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13))
    AND useragent.is_bot = false 
--limit to only sessions with searches submitted via the button
    AND event.action = 'click'
    AND event.position = -1
    AND event.source = 'autocomplete'
    AND event.subTest IS NULL 
    GROUP BY event.searchsessionID,
    session_id_noinput
) all_sessions
"

In [142]:
noinput_search_click_sessions <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [144]:
noinput_search_click_sessions_prop <- noinput_search_click_sessions  %>%
    mutate(prop_search_sessions = blank_search_sessions/all_search_click_sessions * 100)

noinput_search_click_sessions_prop

blank_search_sessions,all_search_click_sessions,prop_search_sessions
<int>,<int>,<dbl>
5073389,49306614,10.28947


In **5.54%** of all search sessions and **10.29%** of search sessions where the search button was clicked, a user clicked the search button (or pressed enter) with no input in the search box. 

# Where are searches conducted (search widget vs advanced search page)?

## What percent of searches are submitted on the Advanced Search Page vs the Search widget?

When a user starts typing in the search widget, the following is recorded: action: "searchResultPage", inputLocation = "header", source = "autocomplete"
When a user starts typing in the advanced search page, the following is recorded: action: "searchResultPage", inputLocation = "content", source = "autocomplete"


In [145]:
# Searches conducted on header (widget) vs Advanced search page
query <- "
SELECT
    SUM(CAST(event.inputLocation = 'header' AS INT)) AS search_widget_events,
    SUM(CAST(event.inputLocation = 'content'  AS INT)) AS adv_search_events,
    COUNT(*) AS all_search_input_events
FROM event.searchSatisfaction
WHERE 
    year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13)) 
--isolate to autocomplete search events - logged when someone starts typing
    AND event.source = 'autocomplete'
    AND event.action = 'searchResultPage'
    AND event.inputLocation IN ('header', 'content')
-- remove bots
    AND useragent.is_bot = false 
-- remove users in any sub test
    AND event.subTest IS NULL
"

In [146]:
search_events_bylocation <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [148]:
search_events_prop_bylocation <- search_events_bylocation %>%
    gather(location_type, num_events, 1:3) %>%
    mutate(prop_search_events = num_events/search_events_bylocation$all_search_input_events * 100) 

search_events_prop_bylocation

location_type,num_events,prop_search_events
<chr>,<int>,<dbl>
search_widget_events,503171135,86.00306
adv_search_events,81890747,13.99694
all_search_input_events,585061882,100.00000


## What percent of search sessions includes searches on Advanced Search Page or Search Widget?

In [149]:
query <-
"
SELECT
    SUM(CAST(search_widget_event = true AS INT)) AS search_widget_sessions,
    SUM(CAST(adv_search_event = true AS INT)) AS adv_search_sessions,
    COUNT(search_session) AS all_sessions
FROM
(
SELECT
    SUM(CAST(event.inputLocation = 'header' AS INT)) >= 1 AS search_widget_event,
    SUM(CAST(event.inputLocation = 'content' AS INT)) >= 1 AS adv_search_event,
    event.searchsessionID AS search_session
FROM event.searchSatisfaction
WHERE 
    year = 2020 
    AND (month <= 07 OR (month =08 AND day <= 13)) 
--isolate to autocomplete search sessions - logged when someone starts typing
    AND event.source = 'autocomplete'
    AND event.action = 'searchResultPage'
    AND event.inputLocation IN ('header', 'content')
-- remove bots
    AND useragent.is_bot = false 
-- remove users in any sub test
    AND event.subTest IS NULL
GROUP BY event.searchsessionID
) AS search
"

In [150]:
search_sessions_bylocation <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [152]:
search_sessions_prop_bylocation <- search_sessions_bylocation %>%
    gather(location_type, num_sessions, 1:3) %>%
    mutate(prop_search_sessions = num_sessions/search_sessions_bylocation$all_sessions * 100) 

search_sessions_prop_bylocation

location_type,num_sessions,prop_search_sessions
<chr>,<int>,<dbl>
search_widget_sessions,72200534,97.23931
adv_search_sessions,7484071,10.07951
all_sessions,74250354,100.00000
